In [2]:
with open("../key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"
MODEL_ID = 'bielik'

In [3]:
import requests
import pandas as pd
import json
import time
import requests

In [4]:
def prompt_chat_custom_temperature(model_id, prompt, temperature = 0.3):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected
    else:
        return None  # Handle non-200 status codes

In [6]:
df = pd.read_csv("../data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [7]:
prompts = [
    "'{english_text}' Przetłumacz na polski: ",
    "Przetłumacz na polski: '{english_text}'",
    "Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'",
    "'{english_text}' Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
    "Przetłumacz '{english_text}' na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",

    "Zdanie do przetłumaczenia: '{english_text}' \n" +
    "Przetłumacz podane zdanie na polski:",

    "Zdanie do przetłumaczenia: '{english_text}' \n" +
    "Przetłumacz podane zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie:",
]

In [8]:
sampled_df = df.sample(n=1000, random_state=50, ignore_index=True)

for prompt_index, prompt_template in enumerate(prompts):

    print(prompt_index, prompt_template)

    for index, row in sampled_df.iterrows():

        english_text = row["en_text"]
        prompt = prompt_template.format(english_text=english_text)

        translation = prompt_chat_custom_temperature(
            MODEL_ID, 
            prompt, 
            0.0
        )
        sampled_df.at[index, f'{MODEL_ID}_prompt{prompt_index}_translation'] = translation
        
        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/bielik_translations_prompt_templates.csv", sep='|', index=False)

0 '{english_text}' Przetłumacz na polski: 
Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...
1 Przetłumacz na polski: '{english_text}'
Processed 100 out of 1000 samples...
Processed 200 out of 1000 samples...
Processed 300 out of 1000 samples...
Processed 400 out of 1000 samples...
Processed 500 out of 1000 samples...
Processed 600 out of 1000 samples...
Processed 700 out of 1000 samples...
Processed 800 out of 1000 samples...
Processed 900 out of 1000 samples...
Processed 1000 out of 1000 samples...
2 Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'
Processed 100 out of 1000 samples...
Processed 200 out o